In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import time
import random

In [34]:
def to_df(file_path):
    with open(file_path, 'r') as f:
        i, df = 0, {}
        for line in f:
            df[i] = eval(line)
            i += 1
        df = pd.DataFrame.from_dict(df, orient='index')
    return df

In [35]:
review_df = to_df('./data/reviews_Electronics_5.json')

In [36]:
meta_df = to_df('./data/meta_Electronics.json')

In [37]:
review_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [38]:
meta_df.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [39]:
print(len(review_df['asin']))
print(len(meta_df['asin']))

24187
20603


In [40]:
meta_df = meta_df[meta_df['asin'].isin(review_df['asin'].unique())]
meta_df_reset = meta_df.reset_index(drop=True)

In [41]:
review_df = review_df[["reviewerID","asin","unixReviewTime"]]
meta_df = meta_df[["asin", "categories"]]
meta_df['categories'] = meta_df['categories'].map(lambda x:x[-1][-1])

In [42]:
print(review_df.head())
print(meta_df.head())
print(len(review_df['asin']))
print(len(meta_df['asin']))

       reviewerID        asin  unixReviewTime
0   AO94DHGC771SJ  0528881469      1370131200
1   AMO214LNFCEI4  0528881469      1290643200
2  A3N7T0DY83Y4IG  0528881469      1283990400
3  A1H8PY3QHMQQA0  0528881469      1290556800
4  A24EV6RXELQZ63  0528881469      1317254400
          asin                   categories
4   0528881469                 Trucking GPS
15  0594451647          Chargers & Adapters
20  0594481813               Power Adapters
38  0972683275     TV Ceiling & Wall Mounts
53  1400532620  eBook Readers & Accessories
24187
971


In [43]:
def build_map(df, col):
    key = sorted(df[col].unique().tolist())
    m = dict(zip(key, range(len(key))))
    df[col] = df[col].map(lambda x:m[x])
    return m, key

In [44]:
asin_map, asin_key = build_map(meta_df, 'asin')
cate_map, cate_key = build_map(meta_df, 'categories')
rev_map, rev_key = build_map(review_df, 'reviewerID')

In [45]:
user_count, item_count, cate_count, example_count = len(rev_map), len(asin_map), len(cate_map), review_df.shape[0]
print('user_count: %d\titem_count: %d\tcate_count: %d\texample_count: %d' %(user_count, item_count, cate_count, example_count))

user_count: 19168	item_count: 971	cate_count: 241	example_count: 24187


In [46]:
print(review_df.head())
print(meta_df.head())
print(len(review_df['asin']))
print(len(meta_df['asin']))

   reviewerID        asin  unixReviewTime
0       17558  0528881469      1370131200
1       17340  0528881469      1290643200
2       13395  0528881469      1283990400
3        2433  0528881469      1290556800
4        5745  0528881469      1317254400
    asin  categories
4      0         217
15     1          47
20     2         167
38     3         206
53     7         240
24187
971


In [47]:
print(sorted(review_df['asin'])[:100])

['0528881469', '0528881469', '0528881469', '0528881469', '0528881469', '0594451647', '0594451647', '0594451647', '0594451647', '0594451647', '0594481813', '0594481813', '0594481813', '0594481813', '0594481813', '0594481813', '0594481813', '0594481813', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972683275', '0972

In [48]:
print(meta_df.head())
meta_df = meta_df.sort_values('asin').reset_index(drop=True)
print(meta_df.head())

    asin  categories
4      0         217
15     1          47
20     2         167
38     3         206
53     7         240
   asin  categories
0     0         217
1     1          47
2     2         167
3     3         206
4     4         210


In [49]:
print(review_df.head())
review_df['asin'] = review_df['asin'].map(lambda x:asin_map[x])
review_df1 = review_df.sort_values(["reviewerID","unixReviewTime"]).reset_index(drop=True)
review_df2 = review_df1[["reviewerID","asin","unixReviewTime"]]
print(review_df2.head())

   reviewerID        asin  unixReviewTime
0       17558  0528881469      1370131200
1       17340  0528881469      1290643200
2       13395  0528881469      1283990400
3        2433  0528881469      1290556800
4        5745  0528881469      1317254400
   reviewerID  asin  unixReviewTime
0           0    75      1385337600
1           1   890      1358035200
2           2   643      1361750400
3           3   168      1390003200
4           4   533      1350086400


In [50]:
print(review_df2.head())
print(meta_df.head())

   reviewerID  asin  unixReviewTime
0           0    75      1385337600
1           1   890      1358035200
2           2   643      1361750400
3           3   168      1390003200
4           4   533      1350086400
   asin  categories
0     0         217
1     1          47
2     2         167
3     3         206
4     4         210


In [51]:
print(sorted(review_df2['asin'])[:100])
print(review_df2['asin'][:20])

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
0      75
1     890
2     643
3     168
4     533
5     303
6     223
7     346
8     455
9     860
10    865
11    666
12    257
13    179
14    220
15    223
16      8
17    123
18    762
19      3
Name: asin, dtype: int64


In [52]:
tmp = review_df2.groupby('reviewerID')
tmp = sorted(tmp['asin'])

In [53]:
print(tmp[:10])

[(0, 0    75
Name: asin, dtype: int64), (1, 1    890
Name: asin, dtype: int64), (2, 2    643
Name: asin, dtype: int64), (3, 3    168
Name: asin, dtype: int64), (4, 4    533
Name: asin, dtype: int64), (5, 5    303
Name: asin, dtype: int64), (6, 6    223
7    346
Name: asin, dtype: int64), (7, 8    455
Name: asin, dtype: int64), (8, 9    860
Name: asin, dtype: int64), (9, 10    865
11    666
12    257
Name: asin, dtype: int64)]


In [54]:
cate_list = np.array([meta_df['categories'][i] for i in range(len(asin_map))],dtype=np.int32)

In [55]:
train_set, test_set = [],[]
for reviewerID, hist in review_df2.groupby('reviewerID'):
    post_list = hist['asin'].tolist()

    def gen_neg():
        neg = post_list[0]
        while(neg in post_list):
            neg = random.randint(0, item_count-1)
        return neg
    neg_list = [gen_neg() for i in range(len(post_list))]
    for i in range(1, len(post_list)):
        hist = post_list[:i]
        
        if(i == len(post_list)-1):
            test_set.append((reviewerID, hist, post_list[i], 1))
        else:
            train_set.append((reviewerID, hist, post_list[i], 1))
            train_set.append((reviewerID, hist, neg_list[i], 0))


In [56]:
import random
random.shuffle(train_set)
random.shuffle(test_set)

In [57]:
class DataInput(object):
    def __init__(self, data, batch_size):
        self.batch_size = batch_size
        self.data = data
#         self.epoch_size = (len(self.data)+self.batch_size-1) // self.batch_size
        self.epoch_size = len(self.data) // self.batch_size
        # 计算迭代次数
        self.epoch_size + 1 if self.epoch_size * self.batch_size <len(self.data) else self.epoch_size
        
        self.idx = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if(self.idx == self.epoch_size):
            raise StopIteration
        start, end = self.idx*self.batch_size, min((self.idx+1)*self.batch_size, len(self.data))
        b_data = self.data[start:end]
        self.idx += 1
        user_id, item_id, y, sample_len = [], [], [], []
        for i in b_data:
            user_id.append(i[0])
            item_id.append(i[2])
            y.append(i[3])
            sample_len.append(len(i[1]))
        max_sl = max(sample_len)
        hist_matrix = np.zeros([len(b_data), max_sl], dtype=np.int64)
        k = 0
        for sample in b_data:
            for j in range(len(sample[1])):
                hist_matrix[k][j] = sample[1][j]
            k += 1
        return self.idx, (user_id, item_id, y, hist_matrix, sample_len)
    

In [58]:
random.seed(1234)
tf.set_random_seed(1234)

train_batch_size =32
test_batch_size = 512


In [59]:
def dice(s, axis=-1, eps=0.001, name=''):
    '''
    PRelu损失函数
    B:batch
    '''
    with tf.variable_scope(name_or_scope='', reuse=tf.AUTO_REUSE):
        alpha = tf.get_variable('alpha'+name, s.get_shape()[-1],                                  
                             initializer=tf.constant_initializer(0.0),                         
                             dtype=tf.float32)
    
    input_shape = list(s.get_shape()) # [1, hidden_size]
    reduction_axis = list(range(len(input_shape))) # [0,1]
    
    del reduction_axis[axis]
    
#     broadcast_shape = [1] * len(input_shape) # [1, 1]
#     broadcast_shape[axis] = input_shape[axis] # [1, hidden_size]
#     mean = tf.reduce_mean(s, axis=reduction_axis) # [hidden_size,]
#     broadcast_mean = tf.reshape(mean, broadcast_shape) # [1, hidden_size]
#     std = tf.reduce_mean(tf.square(s-broadcast_mean), axis=reduction_axis) # [hidden_size,]
#     std = tf.sqrt(std) # [hidden_size]
#     broadcast_std = tf.reshape(std, broadcast_shape) # [1, hiddensize]
#     s_normed = (s-broadcast_mean) / (broadcast_std+eps) # [B, hidden_size]

    s_normed = tf.layers.batch_normalization(s, epsilon=eps, center=False,scale=False)
    
    p = tf.sigmoid(s_normed) # [B, hidden_size]
    
    return alpha*(1.0-p)*s+p*s


In [60]:
def attention(queries, keys, keys_length):
    '''
    queries:     [B, H] item_emb
    keys:        [B, T, H] hist_emb
    keys_length: [B] sample_len
    B:32,batch
    T:用户感兴趣的物品个数的最大值
    H:128,hidden_size
    输入item_emb和用户的hist_emb，输出用户的特征表达
    用户的特征表达用想关的历史物品hist_emb求和表示，此处加了attention赋予不同hist_emb不同权重
    其中权重是和预测的物品相关的函数，预测物品不同，权重分布就不同，权重函数使用DNN实现
    '''
    queries_hidden_units = queries.get_shape().as_list()[-1]
    # shape:0, data:H
    queries = tf.tile(queries, [1, tf.shape(keys)[1]])
    # shape:[B, T*H], data: (item, W)
    queries = tf.reshape(queries, [-1, tf.shape(keys)[1], queries_hidden_units])
    # shape:[B, T, H] data: (item, W)
    din_all = tf.concat([queries, keys, queries-keys, queries*keys], axis=-1)
    # shape:[B, T, 128*4]
    d_layer_1_all = tf.layers.dense(din_all, 80, activation=tf.nn.sigmoid, name='f1_att', reuse=tf.AUTO_REUSE)
    d_layer_2_all = tf.layers.dense(d_layer_1_all, 40, activation=tf.nn.sigmoid, name='f2_att', reuse=tf.AUTO_REUSE)
    d_layer_3_all = tf.layers.dense(d_layer_2_all, 1, activation=tf.nn.sigmoid, name='f3_att', reuse=tf.AUTO_REUSE)
    d_layer_3_all = tf.reshape(d_layer_3_all, [-1, 1, tf.shape(keys)[1]])
    # shape:[B, 1, T]
    outputs = d_layer_3_all
    # shape:[B, 1, T]
    paddings = tf.ones_like(outputs)
    # shape:[B, 1, T]
    key_masks = tf.sequence_mask(keys_length, tf.shape(keys)[1])
    # shape:[B T]
    key_masks = tf.expand_dims(key_masks, 1)
    # shape:[B,1,T]
    outputs = tf.where(key_masks, outputs, paddings)
    outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)
    # (B,1,T) / sqrt(H)， 进行标准化
    outputs = tf.sigmoid(outputs)
    # shape:[B, 1, T]
    outputs = tf.matmul(outputs, keys)
    # (B,1,T) * (B,T,H) ==> (B,1,H)
    # 三维矩阵想乘，相乘发生在后两维
    return outputs

In [63]:
class Model(object):
    def __init__(self, user_count, item_count, cate_count, cate_list):
        # initialization of variables
        self.user = tf.placeholder(tf.int32, [None, ]) # user id, 用户id列表
        self.item = tf.placeholder(tf.int32, [None, ]) # id of items to predict，要预测的物品id列表
        self.item_ = tf.placeholder(tf.int32, [None, ]) # item not like id，不感兴趣的物品id
        self.sl = tf.placeholder(tf.int32, [None, ]) # sample len list，各用户兴趣个数列表
        self.y = tf.placeholder(tf.float32, [None, ]) # label,0/1标签
        self.hist = tf.placeholder(tf.int32, [None, None]) # [Batch, like]，用户兴趣二维列表
        self.lr = tf.placeholder(tf.float64, []) # learning rate，学习率
        hidden_units = 128
        
        with tf.variable_scope('weight', reuse=tf.AUTO_REUSE):
            item_emb_w = tf.get_variable('item_emb_w', [item_count, hidden_units // 2])
            item_emb_b = tf.get_variable('item_emb_b', [item_count], initializer=tf.constant_initializer(0.0))
            cate_emb_w = tf.get_variable('cate_emb_w', [cate_count, hidden_units // 2])
        cate_list = tf.convert_to_tensor(cate_list, dtype=tf.int64)
        ##################################################################
        # get item embedding vectors for input
        
        i_c = tf.gather(cate_list, self.item)  # obtain category of every item
        item_emb = tf.concat(values=[
            tf.nn.embedding_lookup(item_emb_w, self.item), # [B, H/2]
            tf.nn.embedding_lookup(cate_emb_w, i_c)        # [B, H/2]
        ], axis=1) # [B, H]
        ##################################################################
        # get hist embedding vectors for input
        h_c = tf.gather(cate_list, self.hist)
        hist_emb = tf.concat(values=[
            tf.nn.embedding_lookup(item_emb_w, self.hist), # [B, T, H/2]
            tf.nn.embedding_lookup(cate_emb_w, h_c)        # [B, T, H/2]
        ], axis=2) # [B, T, H]
        ##################################################################
        # get user embedding vectors based on user hist vectors and item(to predict) vectors
        att_hist_emb = attention(item_emb, hist_emb, self.sl)       # [B, 1, H]
        att_hist_emb = tf.layers.batch_normalization(inputs=att_hist_emb)  # [B, 1, H]
        att_hist_emb = tf.reshape(att_hist_emb, [-1, hidden_units]) # [B, H]
        att_hist_emb = tf.layers.dense(att_hist_emb, hidden_units)  # [B, H]
        user_emb = att_hist_emb
        ##################################################################
        # MLP
        base_i = tf.concat([user_emb, item_emb], axis=-1) # [B, 2*H]
        base_i = tf.layers.batch_normalization(base_i, name='base_i', reuse=tf.AUTO_REUSE) # [B, 2*H]
        d_layer_1_i = tf.layers.dense(base_i, 80, activation=tf.nn.sigmoid, name='f1', reuse=tf.AUTO_REUSE) # [B, 80]
        d_layer_1_i = dice(d_layer_1_i, name='dice1')
        d_layer_2_i = tf.layers.dense(d_layer_1_i, 40, activation=tf.nn.sigmoid, name='f2', reuse=tf.AUTO_REUSE) # [B, 40]
        d_layer_2_i = dice(d_layer_2_i, name='dice2')
        d_layer_3_i = tf.layers.dense(d_layer_2_i, 1, activation=None, name='f3', reuse=tf.AUTO_REUSE) # [B, 1]
        # 特征平铺
        d_layer_3_i = tf.reshape(d_layer_3_i, [-1]) # [B,]
        i_b = tf.gather(item_emb_b, self.item) # obtain bias of every item, [B,]
        self.pre = i_b+d_layer_3_i # [B]
        # global step
        self.global_step = tf.Variable(0, trainable=False, name = 'global_step')
        self.loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.pre, 
                labels=self.y)
        )
        
        trainable_params = tf.trainable_variables()
        self.opt = tf.train.GradientDescentOptimizer(learning_rate=self.lr)
        gradients = tf.gradients(self.loss, trainable_params)
        clip_gradients, _ = tf.clip_by_global_norm(gradients, 5)
        self.train_op = self.opt.apply_gradients(zip(clip_gradients, trainable_params),
                                                global_step=self.global_step)
    def train(self, sess, item, lr):
        loss, _ =  sess.run([self.loss, self.train_op], feed_dict={
            self.user:item[0],
            self.item:item[1],
            self.y:item[2],
            self.hist:item[3],
            self.sl:item[4],
            self.lr:lr,
        })
        return loss

In [65]:
gpu_options = tf.GPUOptions(allow_growth=True)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    model = Model(user_count, item_count, cate_count, cate_list)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    lr = 0.1
    start_time = time.time()
    for epoch in range(2):
        random.shuffle(train_set)
        loss_sum = 0.0
        for idx, item in DataInput(train_set, train_batch_size):
            loss = model.train(sess, item, lr)
            loss_sum += loss
            if(model.global_step.eval() % 336000 == 0):
                lr  =0.0
        print('epoch: %d, loss: %.4f' % (epoch, loss_sum))
            

epoch: 0, loss: 86.0931
epoch: 1, loss: 85.8102
